---
layout: notes
chapter: 7
chapter-title: Ensemble Learning and Random Forests
permalink: /ml-book/chapter7/notes.html
---

* A group of predictors is called an ensemble
* An ensemble of decision trees is known as a Random Forest
* This so called wisdom of the crowd approach of combining multiple models into an ensemble (or even multi ensembles of models) account for some of the best solutions in Machine Learning
* Popular ensemble methods include bagging, boosting, stacking, and more

## Voting Classifiers
* A voting classifier uses the predictions of multiple diverse predictors (models) and has each one vote and then picks the majority vote
* This is called a hard voting classifier
* You can achieve high accuracy from a bunch of weak learners
* "Suppose you build an ensemble containing 1,000 classifiers that are individually correct only 51% o the time (barely better than random guessing). If you predict the majority voted class, you can hope for up to 75% accuracy!"
* However in practice this is only true if the classifiers are all independent of one another, which isn't the case since they were all trained on the same data
* Ensemble methods work best when the predictors are as independent from one another as possible (e.g. use different training algorithms). This will allow them to make different types of errors, increasing the models accuracy.
* Soft voting is when you use predictors which have the `predict_proba` method (i.e. they predict probabilities). You pick the class with the highest class probability, averaged over all the individual classifiers. This is often more effective because it gives higher weight to higher confidence votes, whereas in hard voting they are all equally weighted (even though some individual models may be unconfident in the result).

## Bagging and Pasting
* Instead of using completely different algorithms to produce an ensemble model, another way to do so is to train the same algorithm on different random subsets of the training set
* This technique is known as bagging and pasting
* Bagging (short for bootstrap aggregating) is when you sample randomly from the training set with replacement
* Pasting is when you sample randomly from the training set without replacement

![image](https://miro.medium.com/max/503/1*z4GrjL9vVGv0PU9Fk1W8zw.png)

* The statistical mode is used for aggregating votes
* This aggregation reduces both bias and variance
* Generally the ensemble has a similar bias, but a lower variance than a single predictor trained on the original training set
* These methods scale very well

### Bagging and Pasting in Scikit-Learn

In [7]:
%%script echo skipping
# What this looks like:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=100, 
                            bootstrap=True, # False would be w/o replacement
                           n_jobs=4)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

Couldn't find program: 'echo'


* Bootstrapping introduces a bit more diversity in the subsets that each predictor is trained on
  * Bagging ends up with a slightly higher bias than pasting, but thisalso means that predictors end up being less correlated so the ensemble's variance is reduced
* Overall, bagging often results in better models, which explains why it is generally preferred
* You can evalute both bagging and pasting if you have time

### Out-of-Bag Evaluation
* https://stats.stackexchange.com/questions/88980/why-on-average-does-each-bootstrap-sample-contain-roughly-two-thirds-of-observat
* https://stats.stackexchange.com/questions/173520/random-forests-out-of-bag-sample-size
* If you sample randomly with replacement it turns out that for each predictors about 37% of the samples will not be selected (for large enough number of samples m)
* These unsampled samples are known as out-of-bag instances
* Thus by performing validation on the oob instances and averaging across predictors we can get an estimate of the accuracy on the test set
* So it serves as like a free validation set `oob_score=True`

## Random Patches and Random Subspaces
* 

Here is the decision boundary created by the tree:

![image](https://user-images.githubusercontent.com/29719483/173130302-13031246-b489-4d2d-a891-088bff582ae3.png)


* Decision Trees are considered a white box model (as opposed to black box) because the it's easy to retrace why decisions were made by the model, whereas in neural network that is not as feasible

## Estimating Class Probabilities
* It assigns probabilities simply based on the fraction of instances for each class that showed up in the associated node in the training set

In [3]:
print(tree_clf.predict_proba([[5, 1.5]]))
print(tree_clf.predict([[5, 1.5]]))

[[0.         0.90740741 0.09259259]]
[1]


## The CART Training Algorithm
* The CART algorithm works as follows:
  * At the root node pick a feature and a threshold for that feature which splits the dataset in half
    * Pick the feature threshold combo which results in the purest subsets (weighted by their size)
  * Continue recursively down the tree until some user defined stop condition is met (i.e. max_depth) or it cannot find an additional split which will reduce the impurity.
* This is a Greedy algorithm in the sense that it starts with an optimal split, and continues to search for it at each level, but it doesn't consider the impurity of lower levels when making a decision for the current level
* In general the optimal tree is NP-Complete $O(exp(m))$ and intractable, in general the CART algorithm produces a reasonably good solution

## Computational Complexity
* It's essential a binary search tree which is $O(log(m))$ to traverse for predictions
* For training, since it compares all features on all samples at each node it is $O(n \times log(m))$

## Gini Impurity or Entropy?
* Entropy is another impurity measure
* A set's entropy is zero when it contains instances of only one class (which is also when gini is 0)

$$H_i = - \sum\limits_{\substack{k=1 \\ p_{i,k} \neq 0}}^n p_{i,k} log(p_{i,k})$$

* **Most of the time it doesn't matter which metric you use. Gini impurity is slightly faster to compute, so it's a good default. When they differ, Gini impurity tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees.**

## Regularization Hyperparameters
* Decision trees left unconstrained are liable to overfit the training data
* They are known as nonparametric models because their parameters are not pre-defined such as in a linear model
* To prevent overfitting, we can use regularization to restrict the degrees of freedom of the decision tree
* There are several regularization parameters to pick from
* Increasing the `min` hyperparameters and reducing the `max` will regularize the model

* Other algorithms work by creating an unrestricted tree and then pruning unnecessary nodes
* Nodes are deemed unnecessary if the children of a node are both end leaves and the p-value dictates that the chances of that split are statiscally insignifcant

![image](https://user-images.githubusercontent.com/29719483/173135440-072bd762-ae06-40d7-9c9c-d56455ecf987.png)

## Regression
* Find an x value that splits the training targets in half and continue splitting until stopping condition, the target value is the mean of the target value of the samples in that node.
* I think this is kind of dumb... like when are results from this sort of regression going to be better... boundaries are only orthogonal

![image](https://user-images.githubusercontent.com/29719483/173137358-b3b5849c-2ea4-4f87-976e-2c6ca0af64dd.png)

## Instability
* Only orthogonal decision boundary which makes them susceptible to rotations in the training data
* Can use PCA to mitigate this issue
* Very sensitive to small variations in the training data
* Removing one sample from the Iris dataset results in a totally different tree, even without removing samples the resultant tree might be different just because of the stochastic nature of the Scikit-learn algorithm
* Random Forests can limit this instability by averaging predictions over many trees